In [83]:
#^ Essential Imports
import logging
import configparser
import pandas as pd
import pandas_ta as ta
import robin_stocks as rstocks
from robin_stocks import robinhood as r
from datetime import datetime
from pytz import timezone
import alive_progress
import traceback
from alive_progress import alive_bar
from icecream import ic
import asyncio
import sys
import time
import os
import random
from tqdm import tqdm
PCT_SPEND = 0.05 # 5% of buying power
verbose_mode = False # Set to True to see more logging output
from ratelimit import limits, sleep_and_retry
from colorama import Fore, Back, Style

In [84]:
#^ load the relevant variables from the .ini credentials file in config/
#^ Starting out with the basics
config = configparser.ConfigParser()
config.read('config/credentials.ini')
coins = config['trading']['coins'].split(', ')
stop_loss_percent = float(config['trading']['stop_loss_percent'])
coins = [coin.strip() for coin in coins]
percent_to_use = float(config['trading']['percent_to_use'])
verbose_mode = config['logging']['verbose_mode']
debug_verbose = config['logging']['debug_verbose']
reset_positions = config['logging']['reset_positions']
minimum_usd_per_position = float(config['trading']['minimum_usd_per_position']) #note: this is the minimum amount of USD that must be invested at any given time in each position. All trades must keep this in mind.
# get pct_to_buy_with from config file
pct_to_buy_with = config['trading']['percent_to_use'] #*fixed
pct_to_buy_with = float(pct_to_buy_with)
pct_to_buy_per_trade = config['trading']['percent_to_spend_per_trade'] #* fixed
pct_to_buy_per_trade = float(pct_to_buy_per_trade)

In [85]:
#^ Logging Setup
if not os.path.exists('logs'):
    os.makedirs('logs')
logging.basicConfig(filename='logs/robinhood.log', level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

In [86]:
"""
Notes
r.crypto.get_crypto_quote(position['currency']['code'])['mark_price']
use the format above to get the current price of a coin in USD

ordering crypto should use this syntax:
    r.orders.order_crypto(
        symbol = coin,
        amountIn = 'dollars', # or 'quantity'
        side = 'buy',
        quantityOrPrice = buy_cost,#buy_cost / float(df.close.iloc[-1]), # this is the amount of the coin to buy in units of the coin
        limitPrice = df.close.iloc[-1],
        timeInForce = 'gtc',
        jsonify = True
    )

Increments apply to each kind of coin
    r.crypto.get_crypto_info('BTC')['min_order_size'] # this is the minimum amount of BTC that can be bought at a time
# Get crypto positions
positions = r.crypto.get_crypto_positions()
print(f'found {len(positions)} positions.')
for position in tqdm(positions):
    # print(position)
    if position['currency']['code'] == crypto:
        pos_dict = position['currency']
        min_order_size = float(pos_dict['increment'])
        coin_holdings = float(position['quantity_available'])
"""

"\nNotes\nr.crypto.get_crypto_quote(position['currency']['code'])['mark_price']\nuse the format above to get the current price of a coin in USD\n\nordering crypto should use this syntax:\n    r.orders.order_crypto(\n        symbol = coin,\n        amountIn = 'dollars', # or 'quantity'\n        side = 'buy',\n        quantityOrPrice = buy_cost,#buy_cost / float(df.close.iloc[-1]), # this is the amount of the coin to buy in units of the coin\n        limitPrice = df.close.iloc[-1],\n        timeInForce = 'gtc',\n        jsonify = True\n    )\n\nIncrements apply to each kind of coin\n    r.crypto.get_crypto_info('BTC')['min_order_size'] # this is the minimum amount of BTC that can be bought at a time\n# Get crypto positions\npositions = r.crypto.get_crypto_positions()\nprint(f'found {len(positions)} positions.')\nfor position in tqdm(positions):\n    # print(position)\n    if position['currency']['code'] == crypto:\n        pos_dict = position['currency']\n        min_order_size = float(p

In [87]:

#! Format for a Buy Order
# buy_order_response = r.orders.order_crypto(
#     symbol = coin,
#     amountIn = 'dollars', # or 'quantity'
#     side = 'buy',
#     quantityOrPrice = buy_cost,#buy_cost / float(df.close.iloc[-1]), # this is the amount of the coin to buy in units of the coin
#     limitPrice = rounded_limit_price,
#     timeInForce = 'gtc',
#     jsonify = True
# )
#! Format for a Sell Order
# # Submit an order using the r.orders.order_sell_crypto_limit() function
# sell_order_response = r.orders.order_crypto(
#     symbol = coin,
#     amountIn = 'quantity', # in coins
#     side = 'sell',
#     quantityOrPrice = volume_to_sell_usd / float(df.close.iloc[-1]), # this is the amount of the coin to sell in units of the coin
#     limitPrice = df.close.iloc[-1],
#     timeInForce = 'gtc',
#     jsonify = True
# )
# r.crypto.get_crypto_quote(position['currency']['code'])['mark_price']
# use the format above to get the current price of a coin in USD


In [88]:
# Purpose: Redo the Robinhood API trader in Python
#* Step One - Log in to the Robinhood API
username = config['robinhood']['username']
password = config['robinhood']['password']
login = r.login(username, password)
print(f"Logged in as {username}")

Logged in as "graham.waters37@gmail.com"


In [89]:
print(f'Checking Available Buying Power...')
total_money = float(r.load_account_profile()['crypto_buying_power']) * pct_to_buy_with
print(f'Available Buying Power: ${total_money}', end='')
print(f' * {pct_to_buy_with} = ${total_money * pct_to_buy_with}')
available_money = total_money * pct_to_buy_with

Checking Available Buying Power...
Available Buying Power: $75.736 * 0.8 = $60.588800000000006


In [95]:
# Action Function -- Buy a Coin
# Input: coin name, Amount in USD, Current Price of Coin
@sleep_and_retry
def get_current_price(coin_name):
    # Get a current quote for the coin in question
    quote = r.crypto.get_crypto_quote(coin_name)
    # Get the current price of the coin
    current_price = quote['mark_price']
    return current_price

@sleep_and_retry
def update_portfolio():
    # Assuming you've already logged in
    positions = r.crypto.get_crypto_positions()
    positions_dict = {}
    for position in positions:
        # print(f"You own {position['quantity']} of {position['currency']['code']}")
        positions_dict[position['currency']['code']] = float(position['quantity'])
    return positions_dict

@sleep_and_retry
def buy_coin(
    coin_name = 'BTC',
    amount = 1.00,
    current_price = 10000
    ):
    # Get the current price of the coin
    current_price = get_current_price(coin_name)
    # Calculate the amount of coins to buy
    amount_to_buy = float(amount) / float(current_price)
    # Buy the coin and return the amount bought (round the buy price to 2 decimal places)
    buying_usd = round(float(amount_to_buy), 2)
    buy_cost = round(float(buying_usd) * float(current_price), 2)
    result = r.orders.order_crypto(
        symbol = coin_name,
        amountIn = 'dollars', # or 'quantity'
        side = 'buy',
        quantityOrPrice = float(buy_cost),#buy_cost / float(df.close.iloc[-1]), # this is the amount of the coin to buy in units of the coin
        limitPrice = float(current_price),
        timeInForce = 'gtc',
        jsonify = True
    )
    return result

@sleep_and_retry
def sell_coin(
    coin_name = 'BTC',
    ):
    positions_dict = update_portfolio()
    # Get a current quote for the coin in question
    quote = r.crypto.get_crypto_quote(coin_name)
    quote_value = quote['mark_price']
    quote_float = float(quote_value)
    quote = round(quote_float, 2)
    # Get the current price of the coin
    current_price = float(get_current_price(coin_name))
    # Calculate the amount of coins to sell
    # Sell All Coins of this type in the portfolio
    amount_held = float(positions_dict[coin_name]) 
    # Sell the coin and return the amount sold (round the sell price to 2 decimal places)
    # selling_usd = round(amount_held * current_price, 2)
    # sell_cost = round(float(selling_usd) / float(current_price), 2) 
    result = r.orders.order_crypto(
        symbol = coin_name,
        amountIn = 'quantity', # or 'quantity'
        side = 'sell', # this is a sell order
        quantityOrPrice = float(amount_held), # this is the amount of the coin to buy in units of the coin
        limitPrice = float(current_price),
        timeInForce = 'gtc', # good till cancelled
        jsonify = True # return json
    )
    return result


basic_orders = {}
positions_dict = update_portfolio()
# amount_held = float(positions_dict[coin]) 
# fill in the basic orders dictionary with 1.00 USD buys of each coin in the coins list
for coin in tqdm(coins):
    basic_orders[coin] = [buy_coin(coin, 1.00, float(get_current_price(coin))), 1.00 / float(get_current_price(coin))] # has the format [amount held, price bought at]
# The calculate_ta_indicators function calculates different technical indicators and generates trading signals based on these indicators. The indicators are: EMA, MACD, RSI,
profit_threshold = 0.10 # 10 cents

# cancel open orders
open_orders = r.orders.get_all_open_crypto_orders()
print(f'You have {len(open_orders)} open orders. Cancelling all open orders.')
for order in tqdm(open_orders):
    r.orders.cancel_crypto_order(order['id'])


run_id = 0 # used to determine if the coin has been held for more than 1 run
def calculate_ta_indicators(coins):
    try:
        print(f'[coin] | Δ % | Δ $ | Δ % (24h) | Δ $ (24h)')
        for coin in coins:
            positions_dict = update_portfolio()
            # coin = 'BTC'  # Replace 'BTC' with the coin you are interested in
            amount_held = float(positions_dict.get(coin, 0)) 
            value_of_held = amount_held * float(get_current_price(coin))
            # update basic_orders dictionary
            previous_value = basic_orders.get(coin, [0, 0])[1]
            basic_orders[coin] = [amount_held, value_of_held]

            delta_success = False
            try:
                # print this
                # [coin] | Δ % | Δ $ | Δ % (24h) | Δ $ (24h) 
                
                delta_percentage = round((value_of_held - previous_value) / previous_value * 100, 4)
                delta_value = round(value_of_held - previous_value, 4)
                if verbose_mode and run_id != 0:
                    print(f'{coin} | {delta_percentage}% | {delta_value} | {delta_percentage}% | {delta_value} |')
                delta_success = True
                run_id += 1
            except:
                pass

            if delta_success and run_id > 1: 
                
                # with all values rounded to 2 decimal places
                if (value_of_held > previous_value + profit_threshold and value_of_held > 1.50) or \
                    delta_percentage > 2.00 or \
                        delta_percentage < -2.00: # if the percent change has either raised by 2% or dropped by 2% sell the coin
                    # sell coin
                    sell_coin(coin)
            elif delta_success == False: # there was an error so go with basic sells
                if (value_of_held > previous_value + profit_threshold and value_of_held > 1.50):
                    # sell coin
                    sell_coin(coin)
            # Get the historical data for the coin
            historicals = r.get_crypto_historicals(coin, interval='hour', span='week', bounds='24_7', info=None)
            
            # Convert the list of dictionaries to a DataFrame
            df = pd.DataFrame(historicals)
            
            # Convert the 'begins_at' column to datetime and set it as the index
            df['begins_at'] = pd.to_datetime(df['begins_at'])
            df.set_index('begins_at', inplace=True)
            
            # Convert the 'close_price' column to float
            df['close_price'] = df['close_price'].astype(float)
            
            # Calculate the EMA
            # print(f'Calculating EMA for {coin}')
            df['ema'] = df['close_price'].ewm(span=12, adjust=False).mean()
            
            # Calculate the MACD
            # print(f'Calculating MACD for {coin}')
            # macd_obj = ta.MACD(df['close_price'])
            df['macd_line'], df['signal_line'], df['macd_hist'] = ta.macd(df['close_price'])
            
            # Calculate the RSI
            # print(f'Calculating RSI for {coin}')
            df['rsi'] = ta.rsi(df['close_price'])
            
            # Buy Signal
            # If the MACD is greater than the signal line and the RSI is less than 30, then buy
            if df['macd_line'].iloc[-1] > df['signal_line'].iloc[-1] and df['rsi'].iloc[-1] < 30:
                buy_coin(coin, 1.00, df['close_price'].iloc[-1])
                # print(f'Buy Signal for {coin}')
            
            # Sell Signal
            # If the MACD is less than the signal line and the RSI is greater than 70, then sell
            elif df['macd_line'].iloc[-1] < df['signal_line'].iloc[-1] and df['rsi'].iloc[-1] > 70:
                sell_coin(coin)
                print(f'Sell Signal for {coin}')
            
            else:
                pass
    except Exception as e:
        print(f'No Signal for {coin}, due to an error {e}')



# use While True to run the program indefinitely
while True:
    calculate_ta_indicators(coins)
    time.sleep(60*5) # wait for 5 minutes




100%|██████████| 14/14 [00:17<00:00,  1.21s/it]


Found Additional pages.


0it [00:00, ?it/s]

You have 0 open orders. Cancelling all open orders.
[coin] | Δ % | Δ $ | Δ % (24h) | Δ $ (24h)


[coin] | Δ % | Δ $ | Δ % (24h) | Δ $ (24h)


# test buy and sell functions
# buy 1.00 USD of BTC
buy_coin(
    coin_name = 'BTC',
    amount = float(1.00),
    current_price = float(r.crypto.get_crypto_quote('BTC')['mark_price'])
    )
print('You bought 1.00 USD of BTC')
# sell 1.00 USD of BTC
sell_coin(
    coin_name = 'BTC'
    )
print('You sold 1.00 USD of BTC')


In [ ]:
# test buy and sell functions
# buy 1.00 USD of BTC

buy_coin(coin = 'BTC', amount = 1.00, 
# sell 1.00 USD of BTC

In [ ]:
COINTELEGRAPH = {
    'ETH': 'https://cointelegraph.com/rss/tag/ethereum',
    'BTC': 'https://cointelegraph.com/rss/tag/bitcoin',
    'XRP': 'https://cointelegraph.com/rss/tag/xrp',
    'BCH': 'https://cointelegraph.com/rss/tag/bitcoin-cash',
    'LTC': 'https://cointelegraph.com/rss/tag/litecoin',
    'XLM': 'https://cointelegraph.com/rss/tag/stellar',
}